In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**WHAT IS DONE HERE?**
Some gentleman (I am grateful to him) has generated here the granular report of the Weather,granular here means the details are gathered day wise.
The components of interest here are Cloud cover,Dew point, Humidity, Precipitation,Wind speed, Maximum temperature, Minimium Temperature, Ozone layer, UV radiation,Confirmed cases and Dead cases. Scope of the data is both Global and particularly United States.

I have analysed data for United States only, for the simple reasons that are

1)Has the geographical land mass that is wide spread and hence one can capture geographical variations.

2)So far has the largest number of casualties. (This is contestable as China may have higher casualties)

After analysing I have tried to relate the COVID-spread with the weather variations of the USA.
Though it may not be possible to uncover the hidden influence of weather (by analysing for US alone), I shall do it for other geographies as well (Europe and India). This shall help us capture the pattern, if one exists.

Since data is day-wise, I have bundled it to month-wise mean.And then have animated it Monthwise.
Matplotlib ArtistAnimation is the most effective tool here.

**Why Matplotlib?**

1)First, I tried with geopandas cartopy and found that mapping can happen Statewise. The data here has the county-level data (like I said granular) and I could not find geopandas doing it (I may be wrong,plz message me in comment box).

2)The figure factory choropleth is deprecated.

3)plotly express choropleth is the best tool here, BUT! the animated file is overwhelmingly large (200MB for a single run), the web browser or the notebook will crash if one goes for it, I strongly say DON't go for it.

4)So what's done is getting the indvidual maps using px.choropleth and then animating them using Matplotlib ArtistAnimation.
  I am open to better ways of doing it as well!


*CODE STRUCTURE*

1)Import the files,

2)Preprocess them (drop the na values, modify the FIPS stype to string and extract the mean monthwise)

3)Plot the data for the counties animated monthwise.

4)Plot bargraph/linegraph for monthwise, aggregated over the entire landscape.

5)Draw the conclusions.

While drawing the conclusions please take in to consideration regionwise population distribution in USA, because thicker populated areas are likely to be affected more than the others.
Also get the idea of population distribution of USA statewise.


*Note: Plz do not let my English spelling skills be the measure of my intelligence,Thank you !!*

<a>Table of Contents</a>


1)[USA population density map](#USA_map)

2)[Cloud Cover Animation](#cloud_cover)

3)[Dew Point Animation](#dew_point)

4)[Humidity Animation](#humidity)

5)[Precipitation Animation](#precipitation)

6)[Wind Speed Animation](#wind)

7)[Temperature Max Animation](#Tmax)

8[Temperature Min Animation](#Tmin)

9)[Ozone Layer and UV Radiation Animation](#ozone)

10)[Confirmed cases Animation](#confirmed)

11)[Casualties Animation](#deaths)

13)[Conclusions](#conclusions)

> Import the Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation  

import geopandas
import calendar
from datetime import datetime
import descartes

import plotly.express as px
import seaborn as sns

%matplotlib notebook

import matplotlib.image as mgimg
from matplotlib import animation
from matplotlib.pyplot import figure

import geopandas
import requests

%matplotlib notebook

In [ ]:
conda install -c anaconda beautifulsoup4


In [ ]:
from bs4 import BeautifulSoup

> 

In [ ]:
#Getting the Data from Wikipedia
wikiurl="https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
soup = BeautifulSoup(response.text, 'html.parser')
pop_table=soup.find('table',{'class':"wikitable"})

#Extracting the Dataframe of States and Population
pop_df=pd.read_html(str(pop_table))
pop_df=pd.DataFrame(pop_df[0])

col1=pop_df['State or territory']
col2=pop_df['% of the total U.S. pop.[b]']['Est. 2020']
col1['percentage']=col2
col1.reset_index(inplace=True)

col1.columns=['Index','NAME','percentage']


<a id='USA_map'>Population Density Map</a>

OBSERVATION From the below graphs 

1)We get the idea of which states are population dense

2)California and Texas alone account for 19% of population. 

3)Coastal states are more dense (Florida, NewYork, California and Washington),continental regions are very sparse 

4)Population towards the Atlantic Coast is denser than any other.

Population density is important because viral contagion is  expected to be thick in population dense areas.


In [ ]:

#Read the Shape file
#One can download the shape files from here, If you are using in Kaggle, load all the 4 files to avoid a certain error
states = geopandas.read_file('../input/shape-files/cb_2018_us_state_5m.shp')
states_plot=states.to_crs("EPSG:3395")

plot_data=pd.merge(states_plot,col1)
plot_data_drp=plot_data.drop([47,53,46,27])

plot_data_drp['percentage'] = plot_data_drp['percentage'].apply(lambda x:float(x.split("%")[0]))
plot_data_drp.plot(column='percentage', cmap='PuRd', k=20,legend=True,figsize=(10,6))
plt.title("Population Density")

In [ ]:
sorted_plot=plot_data_drp.sort_values(by=['percentage'])

plt.figure(figsize=(14, 8))
sns.barplot(data=sorted_plot, x='NAME',y='percentage')
plt.xticks(rotation=90)
plt.title("Statewise population")
plt.show

The below function is about Data preprocessing.
Bundling the columns monthwise and getting the mean

In [ ]:
def df_calculate_mean_sum(in_df, shift_col):
    
    jan_=[];feb_=[];mar_=[];apr_=[];may_=[];jun_=[];jul_=[];aug_=[];sep_=[];oct_=[];nov_=[];dec_=[];
    mnth_dict={'jan':jan_, 'feb':feb_, 'mar':mar_,'apr':apr_,'may':may_, 'jun':jun_,'jul':jul_,
            'aug':aug_, 'sep':sep_,'oct':oct_,'nov':nov_, 'dec':dec_}

    mnth_name_list=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']


    mnth_mean_dict={'jan_mean':[], 'feb_mean':[], 'mar_mean':[], 'apr_mean':[], 'may_mean':[],
               'jun_mean':[],'jul_mean':[], 'aug_mean':[],'sep_mean':[],'oct_mean':[], 'nov_mean':[],'dec_mean':[]}

    mnth_mean_list=['jan_mean','feb_mean','mar_mean','apr_mean','may_mean','jun_mean','jul_mean','aug_mean',
               'sep_mean','oct_mean','nov_mean','dec_mean']
    
    
    if(shift_col==False):
        col_list=in_df.columns
        col_list=col_list[11:len(col_list)]
    else:
        col_list=in_df.columns
        col_list=col_list[12:len(col_list)]
            

    for date in col_list:
        mnth=date.split('/')[0]
        mnth_name=calendar.month_name[int(mnth)]
              
    
        if(mnth_name=='January'):
            jan_.append(date)
        elif(mnth_name=='February'):
            feb_.append(date)
        elif(mnth_name=='March'):
            mar_.append(date)
        elif(mnth_name=='April'):
            apr_.append(date)
        elif(mnth_name=='May'):
            may_.append(date)
        elif(mnth_name=='June'):
            jun_.append(date)
        elif(mnth_name=='July'):
            jul_.append(date)
        elif(mnth_name=='August'):
            aug_.append(date)
        elif(mnth_name=='September'):
            sep_.append(date)
        elif(mnth_name=='October'):
            oct_.append(date)
        elif(mnth_name=='November'):
            nov_.append(date)
        elif(mnth_name=='December'):
            dec_.append(date)
    
    for m_name,m_mean in zip(mnth_name_list, mnth_mean_list):
        if(shift_col==False):
            mnth_mean_dict[m_mean]=in_df[mnth_dict[m_name]].mean(axis=1)
        elif(shift_col==True):
            mnth_mean_dict[m_mean]=in_df[mnth_dict[m_name]].sum(axis=1)
            
            
        
    
    in_df_mean=pd.DataFrame.from_dict(mnth_mean_dict) 
    in_df_mean.insert(0,"State", in_df['Province_State'], True)
    in_df_mean.insert(1,"County", in_df['Admin2'], True)
    in_df_mean.insert(2,"FIPS", in_df['FIPS'], True)
    in_df_mean=in_df_mean.round(2)

    
    return(in_df_mean)


To plot the USA map in choropleth we need the geojson files, we can get it from github

In [ ]:
import json
from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

counties["features"][0]

In [ ]:
#Function for express choropleth map
def anime_plot(data_frame, color_map,month,write_name,title_):
    fig = px.choropleth(data_frame,geojson=counties, locations='FIPS', color=month,
                           color_continuous_scale=color_map,
                           range_color=(0, 20),
                           scope="usa",
                           labels={'month_mean':month}
                          )
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    title=title_,
    title_font_family="Times New Roman",
    title_font_color="black",
    title_font_size=40
    )
    fig.write_image(write_name)

In [ ]:
mnths=['jan_mean','feb_mean','mar_mean','apr_mean','may_mean','jun_mean','jul_mean','aug_mean','sep_mean','oct_mean','nov_mean','dec_mean']
title_list=['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER']
img_list=['jan.png', 'feb.png', 'mar.png', 'apr.png', 'may.png', 'jun.png', 'jul.png', 'aug.png', 'sep.png','oct.png', 'nov.png', 'dec.png']

In [ ]:
conda install -c plotly plotly-orca


In [ ]:
#Get the cloud cover data first
df_cloud=pd.read_csv("../input/weatherclimate-data-covid19/csv/cloud_US.csv", low_memory=False, dtype={"FIPS": str})

df_cloud.dropna(how="any", inplace=True)

df_cloud['FIPS'] = df_cloud['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

cloud_mean=df_calculate_mean_sum(df_cloud,False)


In [ ]:
#Generate the .png choropleth files and dump them in output
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="cloud"+"_"+img_name
    anime_plot(cloud_mean, 'teal', month,img_name,title_)

<a id='cloud_cover'>Cloud Cover in Percentage</a>

(Run the below code to get animation)
The Animation below shows how cloud cover varies across United States Seasonally.


1)Eastern part is mostly cloud covered all the year.

2)The South West coast and the North-west part are the ones that experience fluctuations.

3)Clear skies appear after March and continue till september.

4)August being the month whence most of the landscape has clear skies.

**Seasons in North America**
Winter: December 1 to February 28.

Spring:March 1 to May 31.

Summer: June 1 to August 31.

Fall: September 1 to November 30.

Variations in weather variables should be observed with Seasonal changes in background.



In [ ]:
#The code reads the images from output folder and animates them monthwise.

## Read in graphs

fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="cloud"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
#Get the Dew Point data
df_dew=pd.read_csv("../input/weatherclimate-data-covid19/csv/dew_US.csv", low_memory=False, dtype={"FIPS": str})

df_dew.dropna(how="any", inplace=True)

df_dew['FIPS'] = df_dew['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

dew_mean=df_calculate_mean_sum(df_dew,False)


In [ ]:
#Generate the .png choropleth files and dump them in output
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="dew"+"_"+img_name
    anime_plot(dew_mean, 'jet', month,img_name,title_)

<a id='dew_point'>Dew Point in Temperature Degree/Fahrenheit units</a>

(Run the below code for animation)

Dew point is temperature at which vapor condenses to become water.
It is in a way measure of humidity, higher the humidity higher the dew point (water condenses at nearly room temperatures).
If water vapor is too sparse, needs far lower temperatures to condense.

July,August (peak summers) is when moisture is very high.
The side on Atlantic coast shows observable variations.
Most of the continental part is in frost point (Dry air).

***Virology***: Generally moist weathers favor viral spread.

In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="dew"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
df_hum=pd.read_csv("../input/weatherclimate-data-covid19/csv/humidity_US.csv", low_memory=False, dtype={"FIPS": str})

df_hum.dropna(how="any", inplace=True)

df_hum['FIPS'] = df_hum['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

hum_mean=df_calculate_mean_sum(df_hum,False)


In [ ]:
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="hum"+"_"+img_name
    anime_plot(hum_mean, 'balance', month,img_name,title_)

<a id='humidity'>Humidity</a>

(Run the below code for animation)

Suprisingly humidity is almost a constant factor throughout the year.
The South West Coast and the adjoint strecthes show observable changes as seasons roll.

Make note that though dew point shows considerable variation humidity is nearly constant.

**Virology**: Humidity favors the virus. Since it is constant, its analysis in relation to viral spread is imperceptible.

In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="hum"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
df_precip=pd.read_csv("../input/weatherclimate-data-covid19/csv/precip_US.csv", low_memory=False, dtype={"FIPS": str})

df_precip.dropna(how="any", inplace=True)

df_precip['FIPS'] = df_precip['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

precip_mean=df_calculate_mean_sum(df_precip,False)

In [ ]:
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="precip"+"_"+img_name
    anime_plot(precip_mean, 'geyser', month,img_name,title_)

<a id='precipitation'>Precipitation in length units (cm/mm)</a>
(Run the below code for animation)

1)Most of the Eastern Part recieved precipitation the entire year. So in summers its rainfall and in winters it is snowfall.

2)March is when whole of USA recieves rainfall.

3)South West coast and adjoint strectches are the driest portions of land. August and September being the driest months.

**Virology**: Water is favorable medium for epidemic. Summer rainfalls are very dangerous. Winter snowfall will be detrimental to virus(lower temperatures halt the metabolism of virus)

In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="precip"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
df_wind=pd.read_csv("../input/weatherclimate-data-covid19/csv/wind_US.csv", low_memory=False, dtype={"FIPS": str})

df_wind.dropna(how="any", inplace=True)

df_wind['FIPS'] = df_wind['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

wind_mean=df_calculate_mean_sum(df_wind,False)

In [ ]:
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="wind"+"_"+img_name
    anime_plot(wind_mean, 'plasma', month,img_name,title_)

<a id='wind'>Wind Speed in m/s</a>
(Run the below code for animation)

1)Coasts both East and West maintain a uniform Wind speed.

2)Continental part (the stretch along Texas, Nebraska, Kansas, Dakotas) is windy.

3)The time window between October to December shows variations.

The Winds called WESTERLIES (blow from west to east) flow over the the American landmass.

**Virology**: Wind is the medium through which virus can travel between geographies. As wind flows eastward, rise in number of cases is expected in the East as the winds arrive there. The same time nature of wind (dry or humid) is also a factor. Winds humid and fast aid the virus.


In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="wind"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


Down here we observe all the weather variations (cloud cover, dew point,humidity,precipitation and wind speed ) together.

In [ ]:
cloud_list=[]
dew_list=[]
hum_list=[]
precip_list=[]
wind_list=[]

for month in mnths:
    cloud_list.append(cloud_mean[month].mean())
    dew_list.append(dew_mean[month].mean())
    hum_list.append(hum_mean[month].mean())
    precip_list.append(precip_mean[month].mean())
    wind_list.append(wind_mean[month].mean())
    
line_dict={'Months':title_list,'Cloud_Cover':cloud_list, 'Dew_Point':dew_list, 'Humidity':hum_list, 'Precipitation':precip_list,
          'Wind_Speed':wind_list}

line_frame=pd.DataFrame(line_dict)

fig = plt.figure(figsize=(12,6))

plt.plot(line_frame['Months'], line_frame['Cloud_Cover'], label='Cloud Cover' )
plt.plot(line_frame['Months'], line_frame['Dew_Point'], label='Dew Point' )
plt.plot(line_frame['Months'], line_frame['Humidity'], label='Humidity' )
plt.plot(line_frame['Months'], line_frame['Precipitation'], label='Precipitation' )
plt.plot(line_frame['Months'], line_frame['Wind_Speed'], label='Wind Speed' )


plt.legend()
plt.xticks(rotation=90)
plt.show()



The above graphs reads for landscape taken in total.
Cloud Cover and Dew point vary seasonally.
Humididty too shows gradual change.

In [ ]:
df_tmax=pd.read_csv("../input/weatherclimate-data-covid19/csv/tMax_US.csv", low_memory=False, dtype={"FIPS": str})

df_tmax.dropna(how="any", inplace=True)

df_tmax['FIPS'] = df_tmax['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

tmax_mean=df_calculate_mean_sum(df_tmax,False)

In [ ]:
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="tmax"+"_"+img_name
    anime_plot(tmax_mean, 'thermal', month,img_name,title_)

<a id='Tmax'>Maximum Temperature Variation in degree/Fahnrenheit</a>

(Run the below code for animation)

1)Temperature variations are in sync with seasons, no surprises here

1)The souther portion California, Texas and Florida) warmer because of proximity to equator.

2)July, August are warmer than other months.

****Virology****: Optimum temperatures favor the virus. Too much cold freezes the viral protein structure and too much heat breaks the viral protein. Generally Viruses are more resistant to high temperatures.





In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="tmax"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
df_tmin=pd.read_csv("../input/weatherclimate-data-covid19/csv/tMin_US.csv", low_memory=False, dtype={"FIPS": str})

df_tmin.dropna(how="any", inplace=True)

df_tmin['FIPS'] = df_tmin['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

tmin_mean=df_calculate_mean_sum(df_tmin,False)

In [ ]:
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="tmin"+"_"+img_name
    anime_plot(tmin_mean, 'bluered', month,img_name,title_)

<a id='Tmin'>Minimum Temperatures</a>

1)Minimum temperature variation is again seasonal.

2)Areas northwards and continental become progressively cooler.

3)January appears to be the coldest month

**Virology**
Cold air is good news for humans. Expect cooler areas to be less effected.
But cooler areas are continental and are sparsely populated, so proportionally lesser cases may occur.



In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="tmin"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
df_oz=pd.read_csv("../input/weatherclimate-data-covid19/csv/ozone_US.csv", low_memory=False, dtype={"FIPS": str})

df_oz.dropna(how="any", inplace=True)

df_oz['FIPS'] = df_oz['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

oz_mean=df_calculate_mean_sum(df_oz,False)

In [ ]:
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="oz"+"_"+img_name
    anime_plot(oz_mean, 'magma', month,img_name,title_)

<a id='ozone'>Ozone layer thickness in Dobson units</a>

(Run the below code for animation)

Consider Ozone-layer and UV radiation together.
Ozone layer changes are not so observable, since the CFCs have been phased out.

Only in the Northern Alaska changes are visible, this is due to ozone depletion near the pole which is an established phenomenon.
Since Alaska is largely un-inhabited, it is of little consequence.

UV radiation recieved over the landscape shows clear variation.
But this is not due to Ozone layer depletion.
The change is largely the effect of seasons, UV radiation increases in summer (increased insolation).
The South-West Coast shows maximum reception and Eastern region is almost unaffected. This is very consistent with our earlier observation that Eastern section is largely cloud covered and South West coast has clear skies.

**Virology**

UV radiation may change the protein structure of virus(mutation), whether will it be for good or bad needs to be seen.
UV rays also effect human body too, whether it affects immune response for good or bad needs to be seen.





In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="oz"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
df_uv=pd.read_csv("../input/weatherclimate-data-covid19/csv/uv_US.csv", low_memory=False, dtype={"FIPS": str})

df_uv.dropna(how="any", inplace=True)

df_uv['FIPS'] = df_uv['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

uv_mean=df_calculate_mean_sum(df_uv,False)

In [ ]:
for month, img_name,title_  in zip(mnths,img_list, title_list):
    img_name="uv"+"_"+img_name
    anime_plot(uv_mean,'tropic', month,img_name,title_)

In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list:
    imgs="uv"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
oz_list=[]
uv_list=[]
tmax_list=[]
tmin_list=[]


for month in mnths:
    oz_list.append(oz_mean[month].mean())
    uv_list.append(uv_mean[month].mean())
    tmax_list.append(tmax_mean[month].mean())
    tmin_list.append(tmin_mean[month].mean())
    
    
line_dict={'Months':title_list,'Ozone_Cover':oz_list, 'UV_Radiation':uv_list, 'MAX_temperature':tmax_list, 'MIN_temperature':tmin_list,
          }

line_frame=pd.DataFrame(line_dict)

fig = plt.figure(figsize=(12,6))

plt.plot(line_frame['Months'], line_frame['MAX_temperature'], label='MAX_temperature' )
plt.plot(line_frame['Months'], line_frame['MIN_temperature'], label='MIN_temperature' )
#plt.plot(line_frame['Months'], line_frame['Ozone_Cover'], label='Ozone_Cover' )
plt.plot(line_frame['Months'], line_frame['UV_Radiation'], label='UV_Radiation' )


plt.legend()
plt.xticks(rotation=90)
plt.show()


Above line plots simply show earlier observed behavior of respective parameters.
Temperature and UV radiation show positive correlation.

In [ ]:
df_conf=pd.read_csv("../input/weatherclimate-data-covid19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv", low_memory=False, dtype={"FIPS": str})

df_conf.dropna(how="any", inplace=True)

df_conf['FIPS'] = df_conf['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

conf_sum=df_calculate_mean_sum(df_conf,True)
conf_sum.drop(['sep_mean','oct_mean','nov_mean','dec_mean'], axis=1,inplace=True)


In [ ]:
#Data for Confirmed and Dead cases avaialbel only till first 8 days of August
title_list_=title_list[0:8]
img_list_=img_list[0:8]
mnths_=mnths[0:8]


In [ ]:

for month, img_name,title_  in zip(mnths_,img_list_, title_list_):
    img_name="conf"+"_"+img_name
    anime_plot(conf_sum,'teal', month,img_name,title_)

<a id='confirmed'>Confirmed cases </a>


Below Animation shows how viral spread grows.
Starting from February it rapidly engulfs entire state and by July has fully ocuupied it.
Contagion grows from coastal states and spreads landward.

Detail analysis has been done in combination with casualties reported just few commands below.


In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list_:
    imgs="conf"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
conf_states=conf_sum.groupby('State').sum().reset_index()
conf_states.columns=['NAME','mnth','mnth','mnth','mnth','mnth','mnth','mnth','mnth']
conf_states_sum=conf_states['mnth'].sum(axis=1)

df_conf_states_sum={'NAME':conf_states['NAME'],'Confirmed_cases':conf_states_sum}
df_conf_states_sum=pd.DataFrame(df_conf_states_sum)
df_conf_states_sum=df_conf_states_sum.sort_values(by=['Confirmed_cases'], ascending=False)

df_conf_states_sum.drop([11,1],inplace=True)
sorted_plot_=sorted_plot[['NAME','percentage']].sort_values(by=['percentage'], ascending=False)




In the below plot take a look at
Top States with Confirmed cases namely New york, California, Florida,Texas, New Jersey, Illinois, Massachussetts,Georgia,Pennysylvania etc.

Top States Population wise California, Texas, Florida, New York, Pennysylvania, Illinois, Ohio, Georgia etc.

As it appears, Viral spread is mostly the effect of population density and not of any weather metrics (DUH!!!!)

In [ ]:

fig = plt.figure(figsize=(16,8))

plt.subplot(1, 2, 2)
plt.bar(height=df_conf_states_sum['Confirmed_cases'],x=df_conf_states_sum['NAME'], color='red')
plt.xticks(rotation=90)
plt.title('Confirmed Cases Statewise')

plt.subplot(1, 2, 1)
plt.bar(height=sorted_plot_['percentage'],x=sorted_plot_['NAME'], color='green')
plt.xticks(rotation=90)
plt.title('Population Statewise')


plt.show()





In [ ]:
df_deaths=pd.read_csv("../input/weatherclimate-data-covid19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv", low_memory=False, dtype={"FIPS": str})

df_deaths.dropna(how="any", inplace=True)

df_deaths['FIPS'] = df_deaths['FIPS'].apply(lambda x:'0'+x.split(".")[0] if(len(x)<=6) else x.split(".")[0] ) 

deaths_sum=df_calculate_mean_sum(df_deaths,True)
deaths_sum.drop(['sep_mean','oct_mean','nov_mean','dec_mean'], axis=1,inplace=True)


In [ ]:
for month, img_name,title_  in zip(mnths_,img_list_, title_list_):
    img_name="dth"+"_"+img_name
    anime_plot(deaths_sum,'picnic', month,img_name,title_)

<a id='deaths'>Casualties reported</a>


In [ ]:
fig = plt.figure(figsize=(12,6))
myimages = []


for img_name in img_list_:
    imgs="dth"+"_"+img_name
    img = mgimg.imread(imgs)
    imgplot = plt.imshow(img, animated=True)
    myimages.append([imgplot])
  
    
animation.ArtistAnimation(fig, myimages,interval=2000, blit=True, repeat_delay=100,repeat=True)


In [ ]:
death_states=deaths_sum.groupby('State').sum().reset_index()
death_states.columns=['NAME','mnth','mnth','mnth','mnth','mnth','mnth','mnth','mnth']
death_states_sum=death_states['mnth'].sum(axis=1)

df_death_states_sum={'NAME':death_states['NAME'],'Casualties':death_states_sum}
df_death_states_sum=pd.DataFrame(df_death_states_sum)
df_death_states_sum=df_death_states_sum.sort_values(by=['Casualties'], ascending=False)

df_death_states_sum.drop([11,1],inplace=True)



In [ ]:
fig = plt.figure(figsize=(16,8))

plt.subplot(1, 2, 1)
plt.bar(height=df_conf_states_sum['Confirmed_cases'],x=df_conf_states_sum['NAME'], color='yellow')
plt.xticks(rotation=90)
plt.title('Confirmed Cases Statewise')


plt.subplot(1, 2, 2)
plt.bar(height=df_death_states_sum['Casualties'],x=df_death_states_sum['NAME'], color='magenta')
plt.xticks(rotation=90)
plt.title(' Casualties Statewise')



plt.show()


<a id='conclusions'>Final Conclusions</a>

In the Deaths reported top states being New York, New Jersey Massachusets, Illinois,Michigan, Pennysylvania, California, Florida, Texas.
In the Confirmed cases top states are New York, California, Florida, Texas, New Jersey, Illinois, Massachusets, Gerogia and Pennysylvania.

1)Just like we observed before viral spread is mostly due to population density. To establish correlation with other weather variables requires data with contradicting observations (where population density becomes non factor).

2)Note that California, Texas and Florida are among the most populated states, also have the most confirmed cases but are pushed to back in casualties. This may be due to performance of authorities there and cooperation of people too. Also one might say that immunogenic responses of people in these states (warm states) is good enogh to defeat the virus, this I can say because New York being a cooler states tops both in confirmed and casualties chart.

3)Viral spread starts from March. I dont think it is safe to conclude here that lives lost were mainly in Summer, because detecting cases involves time lag and there may be plenty unreported cases too. Nevertheless one can definitely say that virus becomes active with onset of summer.

4)Viral behavior is definitely impacted by weather, but since human medical advancement is so much that the weather influences are imperceptible. I will analyse the worldwide data to arrive at better conclusions.

5)One more thing is lockdown was imposed, the reduced human activity halts the epidemic and along with weather this is a factor too! unfortunately we have no data for human activity.

6)It is not entirely accurate to say that viral spread is directly related to populated density. All states Texas, California, New York,Florida are coastal states and hence Port States. Influx of foreign goods (trade),foreign citizens might have aided the pandemic.
















